### Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

#### Neural network architecture

In [ ]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Data Import

In [ ]:
from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
val_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

### Modeling + Hyperparameter Optimization

In [ ]:
from genetic import GeneticTraining

hyperparams = {
    "lr": {"type": "exp", "min": 1e-3, "max": 1e-2},
    "batch_size": {"type": "int", "min": 32, "max": 128},
    "n_epochs": {"type": "int", "min": 4, "max": 8},
}

tuner = GeneticTraining(train_data, val_data, NN)

best_hyperparams = tuner.run_evolution(
    population_size=8,
    num_generations=4,
    hyperparams=hyperparams,
    crossover_rate=0.5,
    mutation_rate=0.2,
    mutation_std_factor=8,
    verbose=1
)

print("Best hyperparameters found:", best_hyperparams)

In [ ]:
import pandas as pd

df = pd.DataFrame(list(tuner.fitness_map.values()))

df.sort_values('val_accuracy', ascending=False)